# **Worksapce set up**

In [1]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# Import libraries
%pylab inline
!pip install -q tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras.optimizers import SGD
import numpy as np
import tempfile
import os

Populating the interactive namespace from numpy and matplotlib
     |████████████████████████████████| 237 kB 8.8 MB/s 


In [3]:
# Display python and library versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.21.6
TensorFlow 2.8.2
Keras 2.8.0


# **Dataset Management**

In [4]:
# Loads the data and splits it into 60% training and 40% testing sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_test_old = y_test

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

x_train = x_train[0:6000]
x_test = x_test[0:1000]
y_train = y_train[0:6000]
y_test = y_test[0:1000]

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
# Loads the base model for ANN tests and gets baseline accuracy for result comparison
base_model = tf.keras.models.load_model('drive/MyDrive/GE_practicum/CNN_base')
base_model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 8, 8, 32)         0         
 g2D)                                                            
                                                                 
 flatten_13 (Flatten)        (None, 2048)              0         
                                                                 
 dense_26 (Dense)            (None, 30)                61470     
                                                                 
 dense_27 (Dense)            (None, 10)                310       
                                                                 
Total params: 62,100
Trainable params: 62,100
Non-trainable params: 0
_________________________________________________

# **Quantization and conversion to tflite**

In [6]:
# Conversion to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(base_model)
converter.target_spec.supported_ops = [
tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('CNN_quant.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp22qcymrp/assets


66800

In [7]:
# Check tflite model characteristics
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the CONV_2D op takes
tensor #0 and tensor #7 and tensor #3 as input and produces tensor #8 as output.

Subgraph#0 main(T#0) -> [T#13]
  Op#0 CONV_2D(T#0, T#7, T#3) -> [T#8]
  Op#1 MAX_POOL_2D(T#8) -> [T#9]
  Op#2 RESHAPE(T#9, T#4) -> [T#10]
  Op#3 FULLY_CONNECTED(T#10, T#5, T#2) -> [T#11]
  Op#4 FULLY_CONNECTED(T#11, T#6, T#1) -> [T#12]
  Op#5 SOFTMAX(T#12) -> [T#13]

Tensors of Subgraph#0
  T#0(serving_default_conv2d_13_input:0) shape_signature:[-1, 28, 28, 1], type:FLOAT32
  T#1(dense_27/bias) shape:[10], type:FLOAT32 RO 40 bytes
  T#2(dense_26/bias) shape:[30], type:FLOAT32 RO 120 bytes
  T#3(conv2d_13/bias) shape:[32], type:FLOAT32 RO 128 bytes
  T#4(sequential_13/flatten_13/Const) shape:[2], type:INT32 RO 8 bytes
  T#5(sequential_13/dense_26/MatMul) shape:[30, 2048], type:INT8 RO 61440 bytes
  T#6(sequential_13/dense_

In [8]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()

interpreter.allocate_tensors()
output_details = interpreter.get_output_details()

#Predictions from TFLite model
tfl_pred = []
tfl_pred_class = []
for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]["index"], x_test.astype('float32')[i:i+1,:])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    tfl_pred.append(result)
    tfl_pred_class.append(argmax(result))

right_pred = [y_test_old[i] == tfl_pred_class[i] for i in range(len(y_test))]
acc = sum(right_pred)/len(right_pred)
print(acc)

0.958


# **Conversion to C array**

In [9]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [10]:
# Write TFLite model to a C source (or header) file
with open("CNN_quant" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "CNN_quant"))